## Experiments to verify analytical expression for MSE of Avg Predictor

In [2]:
import numpy as np


In [ ]:
n = 5

#generate a random nxn gaussian matrix
random_matrix = np.random.randn(n, n) #generate a Gaussian random square matrix
A, R = np.linalg.qr(random_matrix) #get the QR decomposition

Pi = np.eye(n)/n